In [ ]:
import energyflow as ef
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = ef.zjets_delphes.load("Herwig", num_data=-1, pad=True, cache_dir='../data',
                                       source='zenodo', which='all',
                                       include_keys=None, exclude_keys=None)

In [ ]:
det_particles = data['sim_particles']
gen_particles = data['gen_particles']